In [37]:
import os
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
import itertools
import ast
import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [38]:
def auto_sig(size, watch_cases, watch_idxs):
    record = pd.DataFrame(columns=['try id', 'Real picks', 'Sham picks', 'p value', 'Judge'])
    all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
    all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
    real_comb = [list(comb) for comb in itertools.combinations(all_real, size)]
    sham_comb = [list(comb) for comb in itertools.combinations(all_sham, size)]

    start_time = time.time()
    try_id = 0

    for real_to_pick in real_comb:
        for sham_to_pick in sham_comb:
            p = fb.auto_compare(real_to_pick, sham_to_pick, watch_cases, watch_idxs)
            record.at[try_id, 'try id'] = try_id
            record.at[try_id, 'Real picks'] = real_to_pick
            record.at[try_id, 'Sham picks'] = sham_to_pick
            record.at[try_id, 'p value'] = p
            record.at[try_id, 'Judge'] = [1 if all(i <= 0.05 for i in sublist if i is not None) else 0 for sublist in p]

            try_id += 1

    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate the difference
    print(f"The program took {elapsed_time} seconds to run.")
    record.to_csv('../../../docs/report/record_'+str(size)+'.csv', index=False)


    return elapsed_time

In [39]:
watch_cases = ['endo', 'exo', 'endo valid', 'exo valid']
watch_idxs = ["RT mean shorten %", "RT std decrease"]
size = 1
# auto_sig(size, watch_cases, watch_idxs)

### View result

In [40]:
record_path = os.path.join('..', '..', '..', 'docs', 'report', 'record_5.csv')
record = pd.read_csv(record_path)

# Use apply with ast.literal_eval to convert string representation of list to actual list
record['Judge'] = record['Judge'].apply(ast.literal_eval)
record['p value'] = record['p value'].apply(ast.literal_eval)

record['p1'] = record['p value'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
record['p2'] = record['p value'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
record['Judge 1'] = record['p1'].apply(lambda x: 1 if sum(i <= 0.05 for i in x) == 3 else 0)
record['Judge 2'] = record['p2'].apply(lambda x: 1 if sum(i <= 0.05 for i in x) == 4 else 0)

# # Filter rows where 'Judge' column is [1, 1]
# record = record[record['Judge'].apply(lambda x: x == [0, 1])]
record = record[record['Judge 1'] == 1]
# record = record[record['Judge 2'] == 1]

# Display the updated DataFrame
display(record)

,try id,Real picks,Sham picks,p value,Judge,p1,p2,Judge 1,Judge 2
2294,2294,"[1, 3, 5, 9, 18]","[2, 6, 10, 11, 15]","[[0.032, 0.016, 0.222, 0.016], [0.548, 1.0, 0....","[0, 0]","[0.032, 0.016, 0.222, 0.016]","[0.548, 1.0, 0.151, 0.548]",1,0
2295,2295,"[1, 3, 5, 9, 18]","[2, 6, 10, 11, 16]","[[0.032, 0.032, 0.548, 0.032], [0.421, 1.0, 0....","[0, 0]","[0.032, 0.032, 0.548, 0.032]","[0.421, 1.0, 0.056, 0.69]",1,0
2298,2298,"[1, 3, 5, 9, 18]","[2, 6, 10, 15, 16]","[[0.032, 0.032, 0.548, 0.032], [0.548, 1.0, 0....","[0, 0]","[0.032, 0.032, 0.548, 0.032]","[0.548, 1.0, 0.151, 0.548]",1,0
2301,2301,"[1, 3, 5, 9, 18]","[2, 6, 11, 15, 16]","[[0.032, 0.032, 0.222, 0.032], [0.548, 1.0, 0....","[0, 0]","[0.032, 0.032, 0.222, 0.032]","[0.548, 1.0, 0.151, 0.841]",1,0
2335,2335,"[1, 3, 5, 9, 18]","[2, 10, 11, 15, 16]","[[0.032, 0.032, 0.548, 0.032], [0.151, 0.548, ...","[0, 0]","[0.032, 0.032, 0.548, 0.032]","[0.151, 0.548, 0.032, 1.0]",1,0
2349,2349,"[1, 3, 5, 9, 18]","[6, 7, 10, 11, 15]","[[0.008, 0.016, 0.222, 0.032], [0.841, 1.0, 0....","[0, 0]","[0.008, 0.016, 0.222, 0.032]","[0.841, 1.0, 0.548, 0.69]",1,0
2370,2370,"[1, 3, 5, 9, 18]","[6, 10, 11, 15, 16]","[[0.008, 0.016, 0.222, 0.016], [0.69, 1.0, 0.1...","[0, 0]","[0.008, 0.016, 0.222, 0.016]","[0.69, 1.0, 0.151, 0.548]",1,0
